## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
gmaps.configure(api_key=g_key)
gmaps.figure()

Figure(layout=FigureLayout(height='420px'))

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/Weather_city.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fare,PF,2021-12-03 01:36:44,-16.7000,-151.0167,80.87,74,49,21.61,light rain
1,1,Hilo,US,2021-12-03 01:32:44,19.7297,-155.0900,80.33,82,75,10.36,light rain
2,2,Carnarvon,AU,2021-12-03 01:36:45,-24.8667,113.6333,75.27,69,90,21.85,overcast clouds
3,3,Souillac,MU,2021-12-03 01:36:45,-20.5167,57.5167,77.38,88,75,6.91,light rain
4,4,Hobart,AU,2021-12-03 01:31:16,-42.8794,147.3294,70.02,44,75,1.01,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 70
What is your desired maximum temperature for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_city_data_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_city_data_df.head()
                            

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fare,PF,2021-12-03 01:36:44,-16.7000,-151.0167,80.87,74,49,21.61,light rain
1,1,Hilo,US,2021-12-03 01:32:44,19.7297,-155.0900,80.33,82,75,10.36,light rain
2,2,Carnarvon,AU,2021-12-03 01:36:45,-24.8667,113.6333,75.27,69,90,21.85,overcast clouds
3,3,Souillac,MU,2021-12-03 01:36:45,-20.5167,57.5167,77.38,88,75,6.91,light rain
4,4,Hobart,AU,2021-12-03 01:31:16,-42.8794,147.3294,70.02,44,75,1.01,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_city_data_df.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
print(clean_city_data_df.shape)

clean_city_data_df.dtypes

(179, 11)


City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Final_city_df = clean_city_data_df.dropna()
Final_city_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
Final_city_df.shape

(178, 11)

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Final_city_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(2)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Fare,PF,80.87,-16.7000,-151.0167,light rain,
1,Hilo,US,80.33,19.7297,-155.0900,light rain,


In [10]:
import pprint

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
#     pprint.pprint(hotels)
#     print(hotels["results"][0]["geometry"]["viewport"]["southwest"]["lat"])
#     break
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(2)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Fare,PF,80.87,-16.7000,-151.0167,light rain,Maitai Lapita Village Huahine
1,Hilo,US,80.33,19.7297,-155.0900,light rain,Hilo Hawaiian Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [17]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))